In [1]:
import pyspark as ps
from collections import Counter
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import random
from data_prep import 
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("nathanscope") 
        .getOrCreate()
        )

In [2]:
sc = spark.sparkContext

In [3]:
col_names = ['marketplace','customer_id','review_id','product_id','product_parent','product_title','product_category','star_rating','helpful_votes','total_votes','vine','verified_purchase','review_headline','review_body','review_date']
cols = {}
for i in range(len(col_names)):
    print (str(i)+': '+col_names[i])
    cols[col_names[i]] = i 

0: marketplace
1: customer_id
2: review_id
3: product_id
4: product_parent
5: product_title
6: product_category
7: star_rating
8: helpful_votes
9: total_votes
10: vine
11: verified_purchase
12: review_headline
13: review_body
14: review_date


In [4]:
def casting_function(row):
    if row[0] == 'marketplace':
        return ()
    if len(row)!=15:
        return(len(row))
    (marketplace, customer_id, review_id, product_id, product_parent, product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date) = row
    
    
    return (marketplace, str(customer_id), str(review_id), str(product_id), product_parent, str(product_title),str(product_category),int(star_rating),int(helpful_votes),int(total_votes),vine,verified_purchase,str(review_headline),str(review_body),review_date)




In [5]:
def time_cast(row):
    pattern = '%Y-%m-%d'
    row =list(row)
    row[14] = int(time.mktime(time.strptime(row[14], pattern)))/86400
    return tuple(row)

In [6]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)

In [7]:
file = open("data/S3_Repos.txt","r+")
repos = []
for line in file.readlines():
    line = line.replace('https://s3.amazonaws.com/amazon-reviews-pds/tsv/','')
    line = line.replace('\n','')
    repos.append('s3://amazon-reviews-pds/tsv/'+line)

In [8]:
repos[39]

's3://amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz'

In [9]:
rdd_books = (sc.textFile(repos[39])
            .map(lambda rowstr : rowstr.split("\t"))
            .map(casting_function)
           )

In [10]:
rdd_books =rdd_books.filter(lambda x: len(x)==15).map(time_cast)

In [11]:
rdd_sample = rdd_books.filter(lambda x: x[7]==1).filter(lambda x: x[9]>10)
#rdd_sample = rdd_books.filter(lambda x: x[7]==1)


star_counts = rdd_books.map(lambda x: Counter({x[cols['star_rating']]:1})).reduce(lambda x, y: x+y)

rdd_sample.count()

In [12]:
tiny_df = pd.DataFrame(rdd_sample.take(10000))
tiny_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,US,12073911,R1KPOIT5WT3NEF,0060839880,66383248,The Case for Hillary Clinton,Books,1,13,29,N,N,hurts to read,"I swear, you've never seen so much manuevering...",13070.0
1,US,12517857,R3AW5JRZYH66M5,097150072X,363879785,Truth vs Falsehood: How to Tell the Difference,Books,1,268,307,N,N,An Unexpected Gift to Hawkins' Readers,"\\""Truth vs Falsehood\\"" is Hawkins' 4th book....",13070.0
2,US,12072415,RA83ZAB84RN88,073561962X,337821525,Introducing Microsoft SQL Server(tm) 2005 for ...,Books,1,10,11,N,N,Obsolete already,Covers only the first beta of Sql 2005. Many ...,13070.0
3,US,24104879,RDBVYHGTTICMV,0307283658,162568542,Harry Potter and the Half-Blood Prince (Book 6),Books,1,4,25,N,N,Always a great reading by Jim Dale,THe book was good as always and Jim Dale consi...,13070.0
4,US,37458619,R35H2BHBL5VNWW,157488221X,932255024,Rain of Ruin: A Photographic History of Hirosh...,Books,1,9,26,N,N,Disappointing,It seems that the people of the United States ...,13070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,US,13929001,R15LTUYIX8F0PS,1401902154,409105258,The Power of Intention,Books,1,29,55,N,Y,The Power of Nothing,Wayne thinks he's the first one to discover wh...,12960.0
9996,US,46799528,RWFIJ0T0P32JX,1590790723,616684281,Michael Jackson: The Man behind the Mask,Books,1,24,28,N,N,"Revenge, and poorly written at that","Having read Jones' book, I can't help but wond...",12960.0
9997,US,24670152,R3MS5KR8BAMEEK,1590790723,616684281,Michael Jackson: The Man behind the Mask,Books,1,10,15,N,N,Disgusting.,In court Bob Jones admitted that this book was...,12960.0
9998,US,17193594,R3LMZTHG3URMCV,0471655856,992167432,Trade Like Jesse Livermore,Books,1,45,47,N,N,DONT WASTE YOUR TIME & MONEY ON THIS BOOK!!!,If you thought you would learn the actual meth...,12960.0


In [13]:
tiny_df.to_csv('data/size_check.csv')

In [14]:
tenK_size = os.path.getsize('data/size_check.csv')

In [15]:
total_rows = rdd_sample.count()
total_rows

143974

In [16]:
optimal_bytes = 90000000
tenK_scale = optimal_bytes/tenK_size
optimal_rows = 10000*tenK_scale


In [17]:
if total_rows <optimal_rows:
    temp = pd.DataFrame(rdd_sample.collect())
else:
    scale = optimal_rows/total_rows
    rdd_sample = rdd_sample.filter(lambda x: random.random() < scale)

In [18]:
total_rows = rdd_sample.count()
total_rows

66864

In [19]:
optimal_rows

66961.54957533773

In [20]:
total_rows

66864

In [21]:
scale

0.46509473637835813

In [ ]:
for repo_name in repos

In [ ]:
rdd_sample.count()

In [ ]:
temp.to_csv('data/books_sample_1star_10vote.csv',index=False)

In [ ]:
new_sample = pd.read_csv('data/books_sample_1star_10vote.csv')

In [23]:
repos

['s3://amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_DVD_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Video_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz',
 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00